In [8]:
# import streamlit as st
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.llms import Ollama

# from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from urllib.parse import urlparse, urlunparse

In [9]:
def format_url(url):
    # Check if the URL has a scheme, if not, assume 'https' and prepend '//' for proper parsing
    if not urlparse(url).scheme:
        url = '//' + url

    # Parse the URL
    parsed_url = urlparse(url, scheme='https')

    # Scheme handling: Ensure the scheme is 'https'
    scheme = 'https'

    # Netloc handling: Directly use the parsed netloc, avoiding adding 'www.' if not appropriate
    netloc = parsed_url.netloc

    # Special handling to avoid adding 'www.' to domains that are already subdomains or include 'www.'
    if not netloc.startswith('www.') and netloc.count('.') == 1:
        netloc = 'www.' + netloc

    # Reconstruct the URL, ensuring only two slashes are used between the scheme and netloc
    formatted_url = urlunparse((scheme, netloc, parsed_url.path, parsed_url.params, parsed_url.query, parsed_url.fragment))

    return formatted_url

In [10]:
loader = WebBaseLoader("https://fundingservice.org.uk")
docs = loader.load()


In [18]:
len(docs[0].page_content)

14359

In [19]:
text_splitter = RecursiveCharacterTextSplitter()
document_chunks = text_splitter.split_documents(docs)

In [22]:
%pip install --upgrade --quiet  gpt4all > /dev/null



Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_community.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings()



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45.9M/45.9M [00:01<00:00, 30.7MiB/s]


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [25]:
text = "This is a test document."

query_result = gpt4all_embd.embed_query(text)



In [44]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

In [46]:
vector_store = Chroma.from_documents(document_chunks, gpt4all_embd)

In [47]:
llm = Ollama(model="mistral")

In [48]:
retriever = vector_store.as_retriever()

prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
    ])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [49]:
prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user's questions based on the below context: \n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
    ])
stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

In [50]:
conversation_rag_chain = create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [56]:
hist=["This is chat history"]

In [53]:
response = conversation_rag_chain.invoke({
        "chat_history": hist,
        "input": "What is this website about?"
    })

In [54]:
print(response['answer'])

 This website appears to be for Energy Funding Service, which is a UK-based organization that helps households access Government Grants to reduce their carbon footprint by installing energy efficient measures such as boilers, solar panels, heat pumps, insulation measures, and IWI. The service assesses the property's eligibility and manages the installation process, with funding available for properties with low energy ratings or high-cost heating. The organization works in partnership with Improveasy and 0800 Repair who are authorized installers for Greater Manchester Combined Authority. They claim that the average property receives between £6,000 – £10,000 in free funding. To check eligibility and apply for funding, individuals can contact them on 0300 302 0221.


In [87]:
response = conversation_rag_chain.invoke({
        "chat_history": hist,
        "input": "tell me key benfits"
    })
print(response['answer'])

 Of using Energy Funding Service to access government grants for energy efficiency improvements? Here are some key benefits:

1. **Cost Savings:** By improving the energy efficiency of your home, you can save money on your energy bills. The specific savings will depend on the measures installed and the size and usage of your property.
2. **Reduced Carbon Emissions:** Energy Funding Service helps reduce carbon emissions by installing energy-efficient measures that use less energy or renewable energy sources. This not only benefits the environment but also contributes to meeting national targets for reducing greenhouse gas emissions.
3. **Free Installation and Funding:** Energy Funding Service assesses, qualifies, and installs eligible energy efficiency measures free of charge, using government grants to cover the costs. There is typically no need for a contribution from the property owner.
4. **Wide Range of Measures:** The range of energy efficiency measures available through Energy Fu

In [ ]:
# def return_rag_chain(hist,):
    

In [94]:
import panel as pn
import param
def load_db(file, chain_type, k):
    return conversation_rag_chain
class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "data from website"
        self.qa = load_db(self.loaded_file,"stuff", 4)


    def convo():
        pass
    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        print("function called", query)
        if not query:
            print('here')
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"input": query, "chat_history": hist})
        self.chat_history.extend([(query, result["answer"])])
        # self.db_query = result["generated_question"]
        # self.db_response = result["source_documents"]
        self.answer = result['answer'] 
        print("result",self.answer )
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return 
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history') 
    def get_chats(self):
        print("This is called")
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [95]:
# pn.extension()

In [99]:
# cb = cbfs()

# file_input = pn.widgets.FileInput(accept='.pdf')
# button_load = pn.widgets.Button(name="Load DB", button_type='primary')
# button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
# button_clearhistory.on_click(cb.clr_history)
# inp = pn.widgets.TextInput( placeholder='How Can I help you?')

# bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
# conversation = pn.bind(cb.convchain, inp) 

# enter_button = pn.widgets.Button(name="Enter", button_type='primary')
# bound_button_load = pn.bind(cb.convo, enter_button.param.clicks)

# jpg_pane = pn.pane.Image( './img/convchain.jpg')

# # tab0 = pn.Column(
# #     pn.Row(  button_load, bound_button_load),
# # )

# tab1 = pn.Column(
#     # pn.Row(button_load),
#     pn.Row(inp),
#     pn.layout.Divider(),
#     pn.panel(conversation,  loading_indicator=True, height=300),
#     pn.layout.Divider(),
# )
# # tab2= pn.Column(
# #     pn.panel(cb.get_lquest),
# #     pn.layout.Divider(),
# #     pn.panel(cb.get_sources ),
# # )
# # tab3= pn.Column(
# #     pn.panel(cb.get_chats),
# #     pn.layout.Divider(),
# # )
# # tab4=pn.Column(
# #     pn.Row( file_input, button_load, bound_button_load),
# #     pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
# #     pn.layout.Divider(),
# #     pn.Row(jpg_pane.clone(width=400))
# # )
# dashboard = pn.Column(
#     pn.Row(pn.pane.Markdown('# Energy Funding Service Chatbot')),
#     pn.Tabs(('Conversation', tab1))
# )
# dashboard

In [115]:
# import panel as pn
# import time

# # Define function to get response from conversation_rag_chain
# def get_response(input_text):
#     hist = []
#     response = conversation_rag_chain.invoke({
#         "chat_history": hist,
#         "input": input_text
#     })
#     return response['answer']

# # Create input text box and button for UI
# input_text = pn.widgets.TextInput(value='How can I help you?')
# button = pn.widgets.Button(name='Enter')

# # Create output text box for UI
# output_text = pn.widgets.TextAreaInput(value='', styles={'width': '600', 'height': '100px'})

# # Create loading spinner for UI
# loader = pn.widgets.Spinner()

# # Define function to update output text when button is clicked
# def update_output(event):
#     # Show loading spinner while bot is running
#     loader.visible = True

#     # Get user input and response from bot
#     user_input = input_text.value
#     bot_response = get_response(user_input)

#     # Update output text box with bot response
#     output_text.value = bot_response

#     # Hide loading spinner
#     loader.visible = False

# # Attach function to button click event
# button.on_click(update_output)

# # Create panel layout with input, button, output, and loader widgets
# panel = pn.Column(
#     pn.Row(pn.pane.Markdown('# Energy Funding Service Chatbot')),
#     pn.layout.Divider(),
#     # pn.Tabs(('Conversation')),
#     pn.Row(input_text, button),
#     # pn.Row(loader),
#     pn.Row(output_text, styles={'margin-top': '10px','width' : '600'}),
#     pn.layout.Divider()
# )

# # Set panel layout style
# panel.style = {
#     'background-color': '#f5f5f5',
#     'padding': '20px',
#     'border-radius': '5px',
#     'box-shadow': '0px 0px 10px rgba(0, 0, 0, 0.1)'
# }

# # Show panel in Jupyter notebook or app
# panel

In [117]:
from typing import List
from fastapi import FastAPI
from langchain.llms import Ollama
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langserve import add_routes
import uvicorn

In [128]:

chain = ConversationalRetrievalChain.from_llm(llm, retriever)


In [129]:
from langchain.schema.output_parser import StrOutputParser

In [124]:
app = FastAPI(title="LangChain", version="1.0", description="The first server ever!")

add_routes(app, conversation_rag_chain, path="/invoke")
uvicorn.run(app, host="localhost", port=8000)


RuntimeError: asyncio.run() cannot be called from a running event loop